In [5]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import explode, collect_list, size, col, row_number, sort_array

In [6]:
sparkSession = SparkSession.builder.enableHiveSupport().master("local").getOrCreate()

In [7]:
graphPath = "/data/graphDFSample"

In [8]:
reversedGraph = sparkSession.read.parquet(graphPath) \
    .withColumn("friend", explode('friends')) \
    .groupBy("friend") \
    .agg(collect_list("user").alias("users")) \
    .withColumn("users_size", size("users"))

reversedGraph.show(3)

+------+--------------------+----------+
|friend|               users|users_size|
+------+--------------------+----------+
|   148|[65051219, 146311...|         4|
|  5518|          [58573511]|         1|
|  9900|          [36844066]|         1|
+------+--------------------+----------+
only showing top 3 rows



In [9]:
reversedGraph = reversedGraph.select(reversedGraph.friend, \
                                     sort_array(reversedGraph.users).alias("users_sorted"), \
                                     "users_size")
reversedGraph.show(3)

+------+--------------------+----------+
|friend|        users_sorted|users_size|
+------+--------------------+----------+
|   148|[3195315, 1463110...|         4|
|  5518|          [58573511]|         1|
|  9900|          [36844066]|         1|
+------+--------------------+----------+
only showing top 3 rows



In [10]:
reversedGraph = reversedGraph.where(reversedGraph.users_size > '1')
reversedGraph.show(3)

+------+--------------------+----------+
|friend|        users_sorted|users_size|
+------+--------------------+----------+
|   148|[3195315, 1463110...|         4|
| 36538|[5506394, 6170161...|        32|
| 41751|[41811068, 60873111]|         2|
+------+--------------------+----------+
only showing top 3 rows



In [11]:
def serializer(input_array):
    out_array = []
    count = 0
    print(count)
    while count < len(input_array):
        for count_internal in range (count+1, len(input_array)):
            print("count: ", count, "count_int: ", count_internal, ", first: ",array[count], ", second: ",array[count_internal])
            out_array.append((array[count], array[count_internal]))
        count += 1
    return(out_array)

In [ ]:
parse_users_udf = f.udf(parse_user_agent, t.ArrayType(t.StringType()))